<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/UCSD Logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/LANL-logo.png" alt="LANL Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/prowess.png" alt="Prowess Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/wildfire.png" alt="WildFire Logo" width="100"/>
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">Fire-Ready Forests Data Challenge</h1>

# Sprint 3: TLS Generated Metrics

In Sprint 1, you worked with TLS data, exploring its format, size, and extracting a sample to create a small visualization. You also looked at a simple approach to process this data.

TLS has the potential to provide accurate and efficient data on forest conditions, including forest structure, vegetation, and fuel loads. As Pokswinski et al. state:

> *The integration of Terrestrial Laser Scanning (TLS) methods into forest monitoring strategies can cost effectively standardize data collection, improve efficiency, and reduce error, with datasets that can easily be analyzed to better inform management decisions.<sup>1</sup>*

In this notebook, we will present metrics derived from TLS data. Our goal is to expose you to these metrics and think about how this information can be incorporated into your workflow. We will not delve into the technical details of how each metric is produced.

For those interested in learning more about TLS data collection, applications, and processing, please refer to the documents included in the references.

## IntELiMon 

The [Interagency Ecosystem LiDAR Monitoring (IntELiMon)](https://dmsdata.cr.usgs.gov/lidar-monitoring/viewer/) is a dashboard for users to access the processed TLS data, metadata, and the results of the processing and analyses. Quality control/quality assurance and metadata management is built into the data workflow.<sup>2</sup>

IntELiMon offers a series of metrics for each of the available plots. These metrics have been [registered in the NDP catalog](https://nationaldataplatform.org/dataset/intelimon-point-cloud-metrics-for-uc-climate-action-seed-project-scaling-science-driven-vegetat) and are presented to you in the following cell.

In [ ]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Loading the metrics
metrics = pd.read_csv("https://wifire-data.sdsc.edu/data/ndp/Data_challenge_2024/new_data/Derived%20TLS%20and%20Tree%20List/metrics/combined_intelimon_metrics.csv")
metrics

As we can observe, there is a lot of information that ha. A complementary document describing each of these metrics is also provided.

In [ ]:
# Loading metrics columns descriptions
metrics_columns = pd.read_csv("https://wifire-data.sdsc.edu/data/ndp/Data_challenge_2024/new_data/Derived%20TLS%20and%20Tree%20List/metrics/IntELiMon_metrics_table_descriptions.csv").iloc[:, :6]
metrics_columns.head()

## Points2Pano

[Points2Pano](https://burnpro3d.sdsc.edu/points2pano/?plot=GAPDR_1071&ts=20220608&m=Basalarea) is an Immersive Forest tool by UCSDs [WIFIRE](https://wifire.ucsd.edu/) lab. RGB colored point clouds submitted to IntELiMon are projected into a spherical raster and displayed on the Points2Pano website. Plots can be viewed by scan date and qualitative comparisons can be made.<sup>2</sup>

You're encouraged to explore this platform.

## TLS Trees vs Field Collected Data

As seen in Points2Pano, TLS can identify trees within a specific range and extract valuable information. In Sprint 1, you were introduced to a treelist collected by a professional team at the UC Climate Action Seed Project study sites. Now, we have a TLS-derived treelist for these same locations.

In the following cells, we will compare tree inventories generated from TLS with field-collected data for the same plots.

In [ ]:
# Loading the full plots inventory
plot_inventory = pd.read_csv("https://wifire-data.sdsc.edu/data/ndp/Data_challenge_2024/new_data/field_data/01_plot_identification.csv")

# Specify our site of interest
site_name = "ind"

# We only keep the inventory plots for Independence Lake
ind_inventory = plot_inventory.loc[plot_inventory.site_name == site_name, ['inventory_id', 'plot_blk']]
ind_inventory

There are a total of 21 blocks associated with Independence Lake. However, for this analysis, we will focus on the first five blocks. We will extract tree data corresponding to these selected plots from both the TLS Treelist and the field-observed dataset.

In [ ]:
# We sample the first 5 plots
ind_sample = ind_inventory.head()

# Load both the TLS and the field collected treelist
tls_treelist = pd.read_csv("https://wifire-data.sdsc.edu/data/ndp/Data_challenge_2024/new_data/Derived%20TLS%20and%20Tree%20List/TLS_treelist.csv")
field_treelist = pd.read_csv("https://wifire-data.sdsc.edu/data/ndp/Data_challenge_2024/new_data/field_data/03_tree.csv")

# Filter tls_treelist based on plot_blk in ind_sample
tls_treelist_ind = tls_treelist.merge(ind_sample[['plot_blk']], on='plot_blk', how='inner')

# Filter field_treelist based on inventory_id in ind_sample
field_treelist_ind = field_treelist.merge(ind_sample[['inventory_id']], on='inventory_id', how='inner')


Now, let's compare the trees from source to the other. Theorically, they are both representing the same sites.

In [ ]:
# Plot trees from both sources
fig, ax = plt.subplots()
plt.scatter(tls_treelist_ind.H, tls_treelist_ind.DBH/2.54, axes = ax, label = 'TLS')
plt.scatter(field_treelist_ind.tree_ht, field_treelist_ind.tree_dbh/10, axes = ax, marker = '.',label= 'Field')
plt.xlabel('Height (m)')
plt.ylabel('DBH (cm)')
plt.legend()
plt.title('TLS vs Field Data')

As you can observe, even when both data sources represent the same sites, the tree inventories they provide can differ significantly.

### References

1. [A simplified and affordable approach to forest monitoring using single terrestrial laser scans and transect sampling.](https://www.sciencedirect.com/science/article/pii/S2215016121002776) Scott Pokswinski, Michael R. Gallagher, Nicholas S. Skowronski, E. Louise Loudermilk, Christie Hawley, Derek Wallace, Alexis Everland, Jon Wallace, J. Kevin Hiers, MethodsX, 2021.
2. [Interagence LiDAR Monitoring and Research Applications - Applications](https://dmsdata.cr.usgs.gov/lidar-monitoring/applications)
3. [Traditional field metrics and terrestrial LiDAR predict plant richness in southern pine forests.](https://www.sciencedirect.com/science/article/pii/S0378112721002073?via%3Dihub)  C.T. Anderson, S.L. Dietz, S.M. Pokswinski, A.M. Jenkins, M.J. Kaeser, J.K. Hiers, B.D. Pelc, Forest Ecology and Management, 2021.
4. [Terrestrial laser scan metrics predict surface vegetation biomass and consumption in a frequently burned southeastern U.S. ecosystem.](https://research.fs.usda.gov/treesearch/66840) E. Louise Loudermilk, Scott Pokswinski, Christie M. Hawley, Aaron Maxwell, Michael Gallagher, Nicholas Skowronski, Andrew T. Hudak, Chad Hoffman, J. Kevin Hiers, bioRxiv, 2023 